# Importamos librerias

In [1]:
import sys

import pandas as pd
import numpy as np
import snowflake.connector
import json
import os
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

    print('Correct Password - connected to SNOWFLAKE')

Connected


# Juli

## Precios Oferta

### tabla 12 - Precios Oferta

In [5]:
url_fechas = 'https://docs.google.com/spreadsheets/d/1JnayuiljvaOebik4gKqNgxSLD6RO2AS8euG7apQykW8/export?format=csv'
fechas = pd.read_csv(url_fechas)
fechas = fechas[pd.to_datetime(fechas['HASTA']) + pd.Timedelta(days=1) >= datetime.today()]
fechas

,EVENTO_ID,DESDE,HASTA,OFERTA
2,NaN,2024-02-15,2024-02-21,OFERTATA FEBRERO 07
3,NaN,2024-02-22,2024-02-28,OFERTATA FEBRERO 08
4,NaN,2024-02-29,2024-03-06,OFERTATA MARZO 09
5,NaN,2024-03-07,2024-03-13,OFERTATA MARZO 10
6,NaN,2024-03-14,2024-03-20,OFERTATA MARZO 11
7,NaN,2024-03-21,2024-03-27,OFERTATA MARZO 12
8,NaN,2024-03-18,2024-03-31,PASCUAS
9,NaN,2024-03-28,2024-04-03,OFERTATA 13
10,NaN,2024-04-04,2024-04-10,OFERTATA 14
11,NaN,2024-02-08,2024-02-14,ESPECIAL CERVEZAS FEBRERO


In [6]:
query = '''
    SELECT
        DISTINCT
        LPTO.*,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        LAA.ORIN,
        LAA.ARTC_ARTC_COD AS ESTADISTICO,
        FP.PROM_PVP_OFERTA
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FP.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    WHERE
        FP.PROM_FECHA_INICIO >= DATEADD(DAY, -30, '{desde_snow}')
        AND FP.PROM_FECHA_FIN < '{desde_snow}'
'''

df_precios_oferta = pd.DataFrame()
for i in range(len(fechas)):
    desde = fechas.iloc[i]['DESDE']
    hasta = fechas.iloc[i]['HASTA']
    oferta = fechas.iloc[i]['OFERTA']

    cursor.execute(query.format(desde_snow = desde))
    df_aux = cursor.fetch_pandas_all()

    df_aux['PRECIO DESDE'] = desde
    df_aux['PRECIO HASTA'] = hasta
    df_aux['NOMBRE_PROMO'] = oferta
    df_precios_oferta = pd.concat([df_precios_oferta, df_aux])

df_precios_oferta = df_precios_oferta[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'NOMBRE_PROMO', 'PRECIO DESDE', 'PRECIO HASTA', 'ORIN', 'ESTADISTICO', 'PROM_PVP_OFERTA']]
print(df_precios_oferta.shape[0])
df_precios_oferta.head(2)

21286


,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,INICIO,FIN,NOMBRE_PROMO,PRECIO DESDE,PRECIO HASTA,ORIN,ESTADISTICO,PROM_PVP_OFERTA
0,12,Otros Mailings,2024-01-25,2024-01-31,OFERTATA FEBRERO 07,2024-02-15,2024-02-21,1000455302,261094000,45.0
1,12,Otros Mailings,2024-02-08,2024-02-14,OFERTATA FEBRERO 07,2024-02-15,2024-02-21,1000435009,253242000,179.0


In [7]:
df_precios_oferta['FIN'] = pd.to_datetime(df_precios_oferta['FIN'])

df_precios_oferta.sort_values(by = ['PRECIO HASTA', 'NOMBRE_PROMO', 'FIN'], ascending = [False, True, False], inplace = True)
df_precios_oferta['FIN'] = df_precios_oferta['FIN'].astype(str)
df_precios_oferta['INICIO'] = df_precios_oferta['INICIO'].astype(str)
df_precios_oferta['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

print(df_precios_oferta.shape[0])
df_precios_oferta.head(2)

21286


,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,INICIO,FIN,NOMBRE_PROMO,PRECIO DESDE,PRECIO HASTA,ORIN,ESTADISTICO,PROM_PVP_OFERTA,FECHA_ACTUALIZACION
0,4,Mailing,2024-03-21,2024-03-27,OFERTATA 14,2024-04-04,2024-04-10,1000455306,140769000,159.0,2024-02-01
3,4,Mailing,2024-03-21,2024-03-27,OFERTATA 14,2024-04-04,2024-04-10,1000456674,163061000,89.0,2024-02-01


## BASE_OPT

### tabla 13 - OPT

In [ ]:
query = '''
    SELECT
        --LAA.ARTC_ARTC_ID,
        --LAA.ARTC_ARTC_COD,
        --LAA.ARTC_ARTC_DESC,
        LAA.ORIN,
        ROUND(AVG(OPT.PRECIO_COMPETENCIA), 1) AS AVG_PRECIO_COMPETENCIA
    FROM
        SPIKE.SPIKE.BASE_OPT AS OPT
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON OPT.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON OPT.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    GROUP BY
        ALL
'''

cursor.execute(query)
opt = cursor.fetch_pandas_all()
opt['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')
opt.head(2)

## Locales Activos Ayer

### tabla 14 - Locales Activos Ayer

In [ ]:
query = '''
SELECT
    --LAA.ARTC_ARTC_DESC AS ARTICULO,
    LAA.ORIN,
    --LAA.ARTC_ARTC_ID,
    COUNT(DISTINCT FS.GEOG_LOCL_ID) AS LOCALES_ACTIVOS_AYER
FROM
    MSTRDB.DWH.FT_STOCK AS FS
    LEFT JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
WHERE
    ARTC_ESTA_ID = 4
    AND LAA.ORIN IS NOT NULL
    AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_LOCL_COD IN (101,102,103,104,107,108,111,115,117,119,120,121,122,124,125,126,127,131,132,134,139,140,141,143,144,145,146,147,148,149,150,151,152,153,155,156,157,158,159,160,161,162,163,164,165,167,166,168,173,198,301,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335))
    AND FS.TIEM_DIA_ID = CURRENT_DATE() - 1
GROUP BY
    ALL
HAVING
    LOCALES_ACTIVOS_AYER > 0
'''

cursor.execute(query)
df_locales_activos_ayer = cursor.fetch_pandas_all()
df_locales_activos_ayer['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')
df_locales_activos_ayer.head(2)

In [ ]:
df_locales_activos_ayer['LOCALES_ACTIVOS_AYER'].nunique()

## Days on Hand

### tabla 15 - days on hand ARTICULO

#### Articulo

In [ ]:
query = '''
WITH STOCK AS
    (
    SELECT
        IM.SUBCLASE,
        LAA.ARTC_ARTC_DESC,
        LAA.ORIN,
        LAA.ARTC_ARTC_ID,
        SUM(FS.STCK_UNIDADES) AS UNIDADES
    FROM
        MSTRDB.DWH.FT_STOCK AS FS
        LEFT JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON LAA.ORIN = IM.ITEM
    WHERE
        FS.TIEM_DIA_ID = CURRENT_DATE() - 1
    GROUP BY
        ALL
    ),

VENTAS AS
    (
    SELECT
        FV.ARTC_ARTC_ID,
        (SUM(FV.VNTA_UNIDADES) / 30) AS UNIDADES_VENDIDAS
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
    WHERE
        FV.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE()-1)
    GROUP BY
        ALL
    )

SELECT
    STOCK.*,
    COALESCE(VENTAS.UNIDADES_VENDIDAS, 0) AS UNIDADES_VENDIDAS
FROM
    STOCK
    LEFT JOIN VENTAS ON VENTAS.ARTC_ARTC_ID = STOCK.ARTC_ARTC_ID
'''

cursor.execute(query)
df_days_on_hand_articulo = cursor.fetch_pandas_all()

df_days_on_hand_articulo = df_days_on_hand_articulo[df_days_on_hand_articulo['UNIDADES'] != 0]
df_days_on_hand_articulo['DAYS ON HAND'] = df_days_on_hand_articulo['UNIDADES'] / df_days_on_hand_articulo['UNIDADES_VENDIDAS']
df_days_on_hand_articulo['DAYS ON HAND'][(df_days_on_hand_articulo['UNIDADES'] == 0) & (df_days_on_hand_articulo['UNIDADES_VENDIDAS'] == 0)] = 0
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] == np.inf] = 999999
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] == -np.inf] = -999999
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] > 999999] = 999999
df_days_on_hand_articulo['DAYS ON HAND'][df_days_on_hand_articulo['DAYS ON HAND'] < -999999] = -999999
df_days_on_hand_articulo['DAYS ON HAND'].fillna(999999, inplace=True)
df_days_on_hand_articulo['DAYS ON HAND'] = round(df_days_on_hand_articulo['DAYS ON HAND'], 0).astype(int)
df_days_on_hand_articulo['UNIDADES'] = round(df_days_on_hand_articulo['UNIDADES'], 0).astype(int)
df_days_on_hand_articulo['UNIDADES_VENDIDAS'] = round(df_days_on_hand_articulo['UNIDADES_VENDIDAS'], 0).astype(int)
df_days_on_hand_articulo.rename({'UNIDADES_VENDIDAS':'UNIDADES VENDIDAS'}, axis = 1, inplace = True)
df_days_on_hand_articulo = df_days_on_hand_articulo[['SUBCLASE', 'ORIN', 'UNIDADES', 'UNIDADES VENDIDAS', 'DAYS ON HAND']]

df_days_on_hand_articulo.rename({'DAYS ON HAND':'DAYS ON HAND ARTICULO'}, axis = 1, inplace = True)

df_days_on_hand_articulo.head(2)

#### Subclase

In [ ]:
query = '''
WITH STOCK AS
    (
    SELECT
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        SUM(FS.STCK_UNIDADES) AS UNIDADES
    FROM
        MSTRDB.DWH.FT_STOCK AS FS
        LEFT JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        FS.TIEM_DIA_ID = CURRENT_DATE() - 1
    GROUP BY
        ALL
    ),

VENTAS AS
    (
    SELECT
        SUB.SUBCLASE,
        (SUM(FV.VNTA_UNIDADES) / 30) AS UNIDADES_VENDIDAS
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        FV.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE()-1)
    GROUP BY
        ALL
    )

SELECT
    STOCK.*,
    COALESCE(VENTAS.UNIDADES_VENDIDAS, 0) AS UNIDADES_VENDIDAS
FROM
    STOCK
    LEFT JOIN VENTAS ON VENTAS.SUBCLASE = STOCK.SUBCLASE
'''

cursor.execute(query)
df_days_on_hand_subclase = cursor.fetch_pandas_all()

df_days_on_hand_subclase = df_days_on_hand_subclase[df_days_on_hand_subclase['UNIDADES'] != 0]
df_days_on_hand_subclase['DAYS ON HAND'] = df_days_on_hand_subclase['UNIDADES'] / df_days_on_hand_subclase['UNIDADES_VENDIDAS']
df_days_on_hand_subclase['DAYS ON HAND'][(df_days_on_hand_subclase['UNIDADES'] == 0) & (df_days_on_hand_subclase['UNIDADES_VENDIDAS'] == 0)] = 0
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] == np.inf] = 999999
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] == -np.inf] = -999999
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] > 999999] = 999999
df_days_on_hand_subclase['DAYS ON HAND'][df_days_on_hand_subclase['DAYS ON HAND'] < -999999] = -999999
df_days_on_hand_subclase['DAYS ON HAND'].fillna(999999, inplace=True)
df_days_on_hand_subclase['DAYS ON HAND'] = round(df_days_on_hand_subclase['DAYS ON HAND'], 0).astype(int)
df_days_on_hand_subclase['UNIDADES'] = round(df_days_on_hand_subclase['UNIDADES'], 0).astype(int)
df_days_on_hand_subclase['UNIDADES_VENDIDAS'] = round(df_days_on_hand_subclase['UNIDADES_VENDIDAS'], 0).astype(int)
df_days_on_hand_subclase.rename({'UNIDADES_VENDIDAS':'UNIDADES VENDIDAS'}, axis = 1, inplace = True)
df_days_on_hand_subclase = df_days_on_hand_subclase[['SUBCLASE', 'UNIDADES', 'UNIDADES VENDIDAS', 'DAYS ON HAND']]

df_days_on_hand_subclase.rename({'DAYS ON HAND':'DAYS ON HAND SUBCLASE'}, axis = 1, inplace = True)

df_days_on_hand_subclase.head(2)

#### Consolido

In [ ]:
df_days_on_hand_articulo = df_days_on_hand_articulo.merge(df_days_on_hand_subclase[['SUBCLASE', 'DAYS ON HAND SUBCLASE']], on = 'SUBCLASE', how = 'left')

df_days_on_hand_articulo['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')

df_days_on_hand_articulo.head(2)

## Precios Stock Mediano ayer

### tabla 16 - Precios Stock Mediano ayer

In [ ]:
query = '''
    SELECT
        DISTINCT
        LPTO.*,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        --LAA.ARTC_ARTC_ID,
        --LAA.ARTC_ARTC_COD,
        --LAA.ARTC_ARTC_DESC,
        LAA.ORIN,
        MEDIAN(FS.STCK_PRECIO_VENTA_DIA_CIVA) AS PRECIO_STOCK_MEDIANO
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FP.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_STOCK AS FS ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID AND FS.TIEM_DIA_ID = CURRENT_DATE - 1
    GROUP BY
        ALL
'''

cursor.execute(query)
df_precios_stock_mediano = cursor.fetch_pandas_all()

df_precios_stock_mediano['FECHA_ACTUALIZACION'] = datetime.today().strftime('%Y-%m-%d')
df_precios_stock_mediano.head(2)

In [ ]:
sys.exit()

# Excel

In [ ]:
with pd.ExcelWriter(f"C:\\Users\\leonardo.mangold\\PycharmProjects\\promos_inteligencia_negocio\\Promos {datetime.today().strftime('%Y-%m-%d')}.xlsx", engine='xlsxwriter') as writer:
    df_precios_oferta.to_excel(writer, sheet_name='Precios Oferta', index=False)
    df_precios_stock_mediano.to_excel(writer, sheet_name='Precios Stock Mediano', index=False)
    opt.to_excel(writer, sheet_name='OPT', index=False)
    df_locales_activos_ayer.to_excel(writer, sheet_name='Locales Activos Ayer', index=False)
    df_days_on_hand_articulo.to_excel(writer, sheet_name='Days on Hand - Articulos', index=False)
    df_days_on_hand_subclase.to_excel(writer, sheet_name='Days on Hand - Subclases', index=False)

# Fin